In [1]:
import os
os.chdir("../..")

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
seed_everything(0)

In [4]:
use_contrastive = True
num_tasks = 5
num_init_tasks = 4
dataset_name = "mnist"
num_epochs = 40

data_cfg = {
    "dataset_name": dataset_name,
    "num_tasks": num_tasks,
    "num_train_per_task": 128,
    "num_val_per_task": 102,
    'remap_labels': True,
    'use_contrastive': use_contrastive,
}
dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [5 0 3 4 2 6 7 1 8 9]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)


In [5]:
net_cfg = {
    'depth': 4,
    'layer_size': 64,
    'num_init_tasks': num_init_tasks,
    'i_size': 28,
    'num_classes': 2,
    'num_tasks': num_tasks,
    'dropout': 0.0,
}

agent_cfg = {
    'memory_size': 64,
    'use_contrastive': use_contrastive,
    'save_dir': 'test',
    "dataset_name": dataset_name,
}

In [6]:
NetCls = MLPSoftLLDynamic
LearnerCls = CompositionalDynamicER
sharing_cfg = DictConfig({
    "scorer": "cross_entropy",
    "num_queries": 4,
    "query_score_threshold": 0.0,
})
train_cfg = {
    # "num_epochs": 40,
    "num_epochs": num_epochs,
    "component_update_freq": num_epochs,
    "num_candidate_modules": 1,
}

agent = Agent(0, 0, dataset,
                NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg, 
                sharing_cfg)

INFO:root:Agent: node_id: 0, seed: 0


In [7]:
## NOTE: there's a bug
## INFO:root:	task: 4	loss: 0.693	acc: 0.491
## for `num_candidate_modules` > 1

for t in range(num_tasks):
    agent.train(t)

INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.692	acc: 0.524
INFO:root:	task: avg	loss: 0.692	acc: 0.524
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.692	acc: 0.524
INFO:root:	task: avg	loss: 0.692	acc: 0.524
INFO:root:final components: 4
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.692	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.493
INFO:root:	task: avg	loss: 0.693	acc: 0.508
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.692	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.493
INFO:root:	task: avg	loss: 0.693	acc: 0.508
INFO:root:final components: 4
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 0.692	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.493
INFO:root:	task: 2	loss: 0.698	acc: 0.481
INFO:root:	task: avg	loss: 0.695	acc: 0.499
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 0.692	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.493
INFO:root:	task: 2	loss: 0.698

NUM_CANDIDATE_MODULES 1


INFO:root:epochs: 1, training task: 4
INFO:root:	task: 0	loss: 0.568	acc: 0.920
INFO:root:	task: 1	loss: 0.597	acc: 0.970
INFO:root:	task: 2	loss: 0.579	acc: 0.951
INFO:root:	task: 3	loss: 0.538	acc: 0.968
INFO:root:	task: 4	loss: 0.694	acc: 0.491
INFO:root:	task: avg	loss: 0.595	acc: 0.860
INFO:root:epochs: 2, training task: 4
INFO:root:	task: 0	loss: 0.568	acc: 0.920
INFO:root:	task: 1	loss: 0.597	acc: 0.970
INFO:root:	task: 2	loss: 0.579	acc: 0.951
INFO:root:	task: 3	loss: 0.538	acc: 0.968
INFO:root:	task: 4	loss: 0.694	acc: 0.491
INFO:root:	task: avg	loss: 0.595	acc: 0.860
INFO:root:epochs: 3, training task: 4
INFO:root:	task: 0	loss: 0.568	acc: 0.920
INFO:root:	task: 1	loss: 0.597	acc: 0.970
INFO:root:	task: 2	loss: 0.579	acc: 0.951
INFO:root:	task: 3	loss: 0.538	acc: 0.968
INFO:root:	task: 4	loss: 0.694	acc: 0.491
INFO:root:	task: avg	loss: 0.595	acc: 0.860
INFO:root:epochs: 4, training task: 4
INFO:root:	task: 0	loss: 0.568	acc: 0.920
INFO:root:	task: 1	loss: 0.597	acc: 0.970
IN

In [8]:
agent.net.structure

ParameterList(
    (0): Parameter containing: [torch.float32 of size 5x4 (GPU 0)]
    (1): Parameter containing: [torch.float32 of size 5x4 (GPU 0)]
    (2): Parameter containing: [torch.float32 of size 5x4 (GPU 0)]
    (3): Parameter containing: [torch.float32 of size 5x4 (GPU 0)]
    (4): Parameter containing: [torch.float32 of size 5x4 (GPU 0)]
)

In [9]:
agent.net.components

ModuleList(
  (0-4): 5 x Linear(in_features=64, out_features=64, bias=True)
)

In [10]:
testloaders = {task: torch.utils.data.DataLoader(testset,
                                                         batch_size=128,
                                                         shuffle=False,
                                                         num_workers=0,
                                                         pin_memory=True,
                                                         ) for task, testset in enumerate(dataset.testset[:num_tasks])}

eval_net(agent.net, testloaders)

{0: 0.9022435897435898,
 1: 0.9748995983935743,
 2: 0.9206030150753769,
 3: 0.957004160887656,
 4: 0.8905698436712053,
 'avg': 0.9290640415542806}